In [1]:
import pandas as pd
import numpy as np

In [2]:
datalake = pd.read_csv('./datalake/datalake.csv', encoding='utf-8', header=0, low_memory=False)

In [3]:
original_data = datalake.copy(deep=True)
print(datalake.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 850013 entries, 0 to 850012
Data columns (total 42 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   COMISARIA           850013 non-null  object
 1   CUADRA              53118 non-null   object
 2   DERIVADA_FISCALIA   715460 non-null  object
 3   DIRECCION           850013 non-null  object
 4   DIST_CIA            850013 non-null  object
 5   DIST_HECHO          850013 non-null  object
 6   DPTO_CIA            850013 non-null  object
 7   DPTO_HECHO          850013 non-null  object
 8   EDAD                847713 non-null  object
 9   ESTADO_DEN          850013 non-null  object
 10  EST_CIVIL           849957 non-null  object
 11  FECHA_HORA_HECHO    850013 non-null  object
 12  ID_COMISARIA        850013 non-null  int64 
 13  ID_EST_CIVIL        849962 non-null  object
 14  ID_LIBRO            850013 non-null  object
 15  ID_MATERIA          850013 non-null  int64 
 16  ID

In [4]:
# Idenficando las columnas por tipo de dato
object_columns = []
int64_columns = []

def update_types():
    global object_columns, int64_columns
    object_columns = datalake.select_dtypes(['object']).columns.to_list()
    int64_columns = datalake.select_dtypes(['int64']).columns.to_list()
    print(f'object ({len(object_columns)})\n {object_columns} ')
    print(f'int64 ({len(int64_columns)})\n {int64_columns} ')

update_types()

object (34)
 ['COMISARIA', 'CUADRA', 'DERIVADA_FISCALIA', 'DIRECCION', 'DIST_CIA', 'DIST_HECHO', 'DPTO_CIA', 'DPTO_HECHO', 'EDAD', 'ESTADO_DEN', 'EST_CIVIL', 'FECHA_HORA_HECHO', 'ID_EST_CIVIL', 'ID_LIBRO', 'ID_NIVEL_EDUCATIVO', 'ID_SIT_PERSONA', 'ID_TIPO_DENUNCIA', 'LIBRO', 'MATERIA', 'MODALIDAD', 'NIVEL_EDUCATIVO', 'OCUPACION', 'PROV_CIA', 'PROV_HECHO', 'REGION', 'SEXO', 'SIT_PERSONA', 'SUB_TIPO', 'TIPO', 'TIPO_DENUNCIA', 'UBICACION', 'VIA', 'fec_registro', 'pais_natal'] 
int64 (8)
 ['ID_COMISARIA', 'ID_MATERIA', 'ID_MODALIDAD', 'ID_REGION', 'ID_SUBTIPO', 'ID_TIPO', 'UBIGEO_CIA', 'UBIGEO_HECHO'] 


In [5]:
#Se están clasificando columnas de id's (incluyendo UBIGEOs) como int64.
#Estos datos no son propiamente numéricos, de hecho, la única categoría numérica sería 'EDAD', pero ahora es de tipo object.
#Primero se convertirán todas las columnas id detectadas como int64 a object
for col in int64_columns:
    datalake[col] = datalake[col].astype(str)
update_types()

object (42)
 ['COMISARIA', 'CUADRA', 'DERIVADA_FISCALIA', 'DIRECCION', 'DIST_CIA', 'DIST_HECHO', 'DPTO_CIA', 'DPTO_HECHO', 'EDAD', 'ESTADO_DEN', 'EST_CIVIL', 'FECHA_HORA_HECHO', 'ID_COMISARIA', 'ID_EST_CIVIL', 'ID_LIBRO', 'ID_MATERIA', 'ID_MODALIDAD', 'ID_NIVEL_EDUCATIVO', 'ID_REGION', 'ID_SIT_PERSONA', 'ID_SUBTIPO', 'ID_TIPO', 'ID_TIPO_DENUNCIA', 'LIBRO', 'MATERIA', 'MODALIDAD', 'NIVEL_EDUCATIVO', 'OCUPACION', 'PROV_CIA', 'PROV_HECHO', 'REGION', 'SEXO', 'SIT_PERSONA', 'SUB_TIPO', 'TIPO', 'TIPO_DENUNCIA', 'UBICACION', 'UBIGEO_CIA', 'UBIGEO_HECHO', 'VIA', 'fec_registro', 'pais_natal'] 
int64 (0)
 [] 


#### Filtro inicial de atributos

In [6]:
# Las columnas ID no se considera necesarias para el análisis, por lo que se eliminan
datalake = datalake[datalake.columns.drop(list(datalake.filter(regex='ID_')))]
# Las columnas UBIGEO identifican el departamento, provincia y distrito de cada registro, por lo que se eliminan
datalake = datalake[datalake.columns.drop(list(datalake.filter(regex='UBIGEO')))]
# Para estandarizar los nombres de las columnas, se convertirán a mayúsculas
datalake.columns = datalake.columns.str.upper()

# La investigación busca predecir la modalidad de violencia familiar que efectua una persona
# Se limita los registros a los que poseen SIT_PERSONA == 'DENUNCIADO'
datalake = datalake[datalake['SIT_PERSONA'] == 'DENUNCIADO']
datalake = datalake.drop('SIT_PERSONA', axis=1)

update_types()
datalake.info()

object (28)
 ['COMISARIA', 'CUADRA', 'DERIVADA_FISCALIA', 'DIRECCION', 'DIST_CIA', 'DIST_HECHO', 'DPTO_CIA', 'DPTO_HECHO', 'EDAD', 'ESTADO_DEN', 'EST_CIVIL', 'FECHA_HORA_HECHO', 'LIBRO', 'MATERIA', 'MODALIDAD', 'NIVEL_EDUCATIVO', 'OCUPACION', 'PROV_CIA', 'PROV_HECHO', 'REGION', 'SEXO', 'SUB_TIPO', 'TIPO', 'TIPO_DENUNCIA', 'UBICACION', 'VIA', 'FEC_REGISTRO', 'PAIS_NATAL'] 
int64 (0)
 [] 
<class 'pandas.core.frame.DataFrame'>
Int64Index: 422522 entries, 0 to 850010
Data columns (total 28 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   COMISARIA          422522 non-null  object
 1   CUADRA             25064 non-null   object
 2   DERIVADA_FISCALIA  349558 non-null  object
 3   DIRECCION          422522 non-null  object
 4   DIST_CIA           422522 non-null  object
 5   DIST_HECHO         422522 non-null  object
 6   DPTO_CIA           422522 non-null  object
 7   DPTO_HECHO         422522 non-null  object
 8   EDAD          

#### Exploración de datos

##### Valores perdidos

In [7]:
# Trimea los textos y si resulta vacio, se pone nan para luego aplicar el fillna
for col in object_columns:
    datalake[col] = datalake[col].str.strip()
datalake = datalake.replace('', np.nan)

In [8]:
# Resumen de valores perdidos
def missing_values():
    row_count = datalake.shape[0]
    print(f'Número total de registros: {row_count}')
    global_null_count = datalake.isnull().sum().sum()
    print(f'Número total de valores perdidos: {global_null_count}')
    print('Valores perdidos por columna:')
    for col in datalake.columns:
        null_count = datalake[col].isnull().sum()
        null_percent = null_count / row_count * 100
        if null_count > 0:
            print(f'{col}: {null_count} ({null_percent:.2f}%)')

missing_values()

Número total de registros: 422522
Número total de valores perdidos: 1365562
Valores perdidos por columna:
CUADRA: 397458 (94.07%)
DERIVADA_FISCALIA: 72967 (17.27%)
EDAD: 5808 (1.37%)
ESTADO_DEN: 172615 (40.85%)
EST_CIVIL: 7 (0.00%)
FECHA_HORA_HECHO: 16091 (3.81%)
NIVEL_EDUCATIVO: 405892 (96.06%)
OCUPACION: 278348 (65.88%)
SEXO: 243 (0.06%)
UBICACION: 13 (0.00%)
FEC_REGISTRO: 16091 (3.81%)
PAIS_NATAL: 29 (0.01%)


##### Valores únicos para cada atributo

In [9]:
def get_uniques(col):
    uniques = datalake[col].dropna().unique()
    print(f'{col} [{len(uniques)} unique values]')
    print(uniques)

In [10]:
for col in datalake.columns:
    get_uniques(col)

COMISARIA [1044 unique values]
['HUANCHACO' 'DULANTO' 'JICAMARCA' ... 'CORDOVA'
 'COMISARIA DE FAMILIA - HUARAZ' 'INCAHUASI']
CUADRA [902 unique values]
['1.0' '2.0' '7.0' '35.0' '12.0' '13.0' '10.0' '6.0' '4.0' '16.0' '17.0'
 '3.0' '15.0' '26.0' '18.0' '5.0' '14.0' '36.0' '8.0' '9.0' '27.0' '22.0'
 '50.0' '38.0' '32.0' '31.0' '11.0' '39.0' '28.0' '24.0' '33.0' '46.0'
 '62.0' '41.0' '23.0' '40.0' '20.0' '37.0' '19.0' '64.0' '30.0' '34.0'
 '70.0' '2732.0' '42.0' '21.0' '25.0' '72.0' '56.0' '356.0' '1532.0' '0.0'
 '154.0' '51.0' '821.0' '374.0' '178.0' '58.0' '501.0' '29.0' '760.0'
 '53.0' '128.0' '650.0' '231.0' '1086.0' '497.0' '227.0' '181.0' '855.0'
 '88.0' '74.0' '1057.0' '44.0' '911.0' '590.0' '830.0' '441.0' '739.0'
 '115.0' '400.0' '83.0' '499.0' '136.0' '979.0' '1379.0' '48.0' '668.0'
 '301.0' '67.0' '470.0' '63.0' '221.0' '126.0' '187.0' '54.0' '245.0'
 '270.0' '273.0' '59.0' '68.0' '1167.0' '3290.0' '466.0' '112.0' '106.0'
 '1034.0' '504.0' '65.0' '505.0' '135.0' '45.0' '385.0

In [11]:
for col in datalake.columns:
    if len(datalake[col].dropna().unique()) == 1:
        print(f'{col} es una columna de tipo único')

MATERIA es una columna de tipo único


In [12]:
# MATERIA contiene un solo valor, por lo que se elimina
datalake = datalake.drop('MATERIA', axis=1)

##### Imputación inicial de valores perdidos

###### Eliminar columnas con alta cantidad de valores nulos

In [13]:
# El porcentaje de cantidad de valores nulos aceptados por columna es 20%
# Se eliminan las columnas con más de 20% de valores nulos
for col in datalake.columns:
    if datalake[col].isnull().sum() / datalake.shape[0] * 100 > 20:
        datalake = datalake.drop(col, axis=1)

In [14]:
missing_values()

Número total de registros: 422522
Número total de valores perdidos: 111249
Valores perdidos por columna:
DERIVADA_FISCALIA: 72967 (17.27%)
EDAD: 5808 (1.37%)
EST_CIVIL: 7 (0.00%)
FECHA_HORA_HECHO: 16091 (3.81%)
SEXO: 243 (0.06%)
UBICACION: 13 (0.00%)
FEC_REGISTRO: 16091 (3.81%)
PAIS_NATAL: 29 (0.01%)


In [15]:
# A este punto todas la columnas son categóricas, incluso EDAD que será transformada después
# Acerca de la imputación de nulos faltante, se usará la moda de las columnas categóricas excepto en las fechas
# Primero se trabajará las columnas de fechas por su formato y luego se trabajarán las demás

##### Valores atípicos

###### Edad

In [16]:
# Llena los valores perdidos con 0
datalake['EDAD'] = datalake['EDAD'].fillna(0)
# Para convertir a int64 primero se convierte a float
datalake['EDAD'] = datalake['EDAD'].astype(float)
datalake['EDAD'] = datalake['EDAD'].astype(np.int64)

In [17]:
# Valores máximos y mínimos
print(datalake['EDAD'].min(), datalake['EDAD'].max())

0 728


In [18]:
# Vista ordenada de edades registradas
age_sorted = datalake.sort_values('EDAD')['EDAD']
print(age_sorted.head())
print(age_sorted.tail())
# Obtención de métricas
median = age_sorted.median()
mode = age_sorted.mode()[0]
print(f'Mediana: {median}')
print(f'Moda: {mode}')

790177    0
162763    0
63659     0
92007     0
323474    0
Name: EDAD, dtype: int64
408108    545
613740    617
176833    617
655314    627
179016    728
Name: EDAD, dtype: int64
Mediana: 34.0
Moda: 31


In [19]:
# Suplantamos los valores atípicos con la moda
# Se considerarán las edades entre 18 y 75 años
min_age = 18
max_age = 75
datalake.loc[(datalake['EDAD'] < min_age) | (datalake['EDAD'] > max_age) | (datalake['EDAD'] == 0), 'EDAD'] = mode
# La columna EDAD ya no tiene valores perdidos

In [20]:
missing_values()

Número total de registros: 422522
Número total de valores perdidos: 105441
Valores perdidos por columna:
DERIVADA_FISCALIA: 72967 (17.27%)
EST_CIVIL: 7 (0.00%)
FECHA_HORA_HECHO: 16091 (3.81%)
SEXO: 243 (0.06%)
UBICACION: 13 (0.00%)
FEC_REGISTRO: 16091 (3.81%)
PAIS_NATAL: 29 (0.01%)


###### Sexo

In [21]:
# SEXO contiene valores extraños
print(datalake['SEXO'].dropna().unique())
# Vemos la frecuencia de los valores
print(datalake['SEXO'].value_counts())
# Solo considerarán los valores M y F, los demás será suplantados por la moda incluyendo los valores extraños
print(f'SEXO: Moda -> {datalake["SEXO"].mode()[0]}')

['M' 'F']
M    340170
F     82109
Name: SEXO, dtype: int64
SEXO: Moda -> M


In [22]:
datalake.loc[(datalake['SEXO'] != 'M') & (datalake['SEXO'] != 'F'), 'SEXO'] = datalake['SEXO'].mode()[0]
print(datalake['SEXO'].value_counts())
# La columna SEXO ya no tiene valores perdidos
missing_values()

M    340413
F     82109
Name: SEXO, dtype: int64
Número total de registros: 422522
Número total de valores perdidos: 105198
Valores perdidos por columna:
DERIVADA_FISCALIA: 72967 (17.27%)
EST_CIVIL: 7 (0.00%)
FECHA_HORA_HECHO: 16091 (3.81%)
UBICACION: 13 (0.00%)
FEC_REGISTRO: 16091 (3.81%)
PAIS_NATAL: 29 (0.01%)


###### Fecha

In [23]:
# Contamos con diferentes formatos de fecha
# mm/dd/yyyy
# yyyy-mm-dd
# MONTH-dd-yyyy
# excel: float

In [24]:
import xlrd

# De fecha de excel (43677.4) a yyyy-mm-dd
def transformExcelDate(excelDate):
    dateISO = xlrd.xldate.xldate_as_datetime(float(excelDate), 0).date().isoformat()
    return dateISO

In [25]:
# RegExp de fecha de excel
import re
excelDate = r'\d+(\.\d+)?'

In [26]:
# Función para estandarizar las fechas de los registros
def transformDate(date):
    try:
        if not pd.isna(date):
            if re.fullmatch(excelDate, date):
                return transformExcelDate(date)
            else:
                return pd.to_datetime(date).date().isoformat()
    except Exception as e:
        print(e, date)

In [27]:
datalake['FEC_REGISTRO'] = datalake['FEC_REGISTRO'].apply(transformDate)
datalake['FECHA_HORA_HECHO'] = datalake['FECHA_HORA_HECHO'].apply(transformDate)

In [28]:
# Los valores perdidos de FEC_REGISTRO y FECHA_HORA_HECHO, oportunamente, comparten registros
# Es decir si en una columna de fecha hay un valor perdido, en la otra también lo hay
datalake = datalake.loc[datalake['FEC_REGISTRO'].isna() == False]
missing_values()

Número total de registros: 406431
Número total de valores perdidos: 68427
Valores perdidos por columna:
DERIVADA_FISCALIA: 68379 (16.82%)
EST_CIVIL: 7 (0.00%)
UBICACION: 12 (0.00%)
PAIS_NATAL: 29 (0.01%)


#### Completando valores perdidos en el resto de atributos categóricos

In [29]:
update_types()

object (22)
 ['COMISARIA', 'DERIVADA_FISCALIA', 'DIRECCION', 'DIST_CIA', 'DIST_HECHO', 'DPTO_CIA', 'DPTO_HECHO', 'EST_CIVIL', 'FECHA_HORA_HECHO', 'LIBRO', 'MODALIDAD', 'PROV_CIA', 'PROV_HECHO', 'REGION', 'SEXO', 'SUB_TIPO', 'TIPO', 'TIPO_DENUNCIA', 'UBICACION', 'VIA', 'FEC_REGISTRO', 'PAIS_NATAL'] 
int64 (1)
 ['EDAD'] 


In [30]:
# Se llena el resto de los valores perdidos con la moda
for c in object_columns:
    mode = datalake[c].mode()[0]
    datalake[c] = datalake[c].fillna(mode)

In [31]:
print(missing_values())

Número total de registros: 406431
Número total de valores perdidos: 0
Valores perdidos por columna:
None


In [32]:
# Reseteamos el index
datalake.reset_index(drop=True, inplace=True)
# Mostramos la tabla
print(datalake.head())
print(datalake.tail())

   COMISARIA   DERIVADA_FISCALIA                                DIRECCION  \
0  HUANCHACO  JUZGADO DE FAMILIA           AV. LA RIVERA PLAZA SAN MARTIN   
1    DULANTO  JUZGADO DE FAMILIA                                   CALLAO   
2  JICAMARCA               OTROS  Mz. Q  S/N Ovalo Principal de Jicamarca   
3      OCOÑA               OTROS                    AV. PANAMERICA SUR SN   
4    PARCONA  JUZGADO DE FAMILIA                             JJ ELIAS S/N   

               DIST_CIA            DIST_HECHO     DPTO_CIA   DPTO_HECHO  EDAD  \
0             HUANCHACO             HUANCHACO  LA LIBERTAD  LA LIBERTAD    32   
1                CALLAO                CALLAO       CALLAO       CALLAO    40   
2  LURIGANCHO - CHOSICA  LURIGANCHO - CHOSICA         LIMA         LIMA    28   
3                 OCOÑA                 OCOÑA     AREQUIPA     AREQUIPA    55   
4               PARCONA               PARCONA          ICA          ICA    21   

     EST_CIVIL FECHA_HORA_HECHO  ... PROV_HECHO   

##### Separando fecha en año, mes, día y día de la semana

In [33]:
def spread_out_date(col):
    datalake[f'{col}_ANIO'] = datalake[col].apply(lambda x: int(x[:4]))
    datalake[f'{col}_MES'] = datalake[col].apply(lambda x: int(x[5:7]))
    datalake[f'{col}_DIA'] = datalake[col].apply(lambda x: int(x[8:]))
    datalake[f'{col}_DIA_SEM'] = datalake[col].apply(lambda x: pd.to_datetime(x).weekday())

In [34]:
spread_out_date('FEC_REGISTRO')
spread_out_date('FECHA_HORA_HECHO')

In [35]:
print(datalake.head())
print(datalake.tail())

   COMISARIA   DERIVADA_FISCALIA                                DIRECCION  \
0  HUANCHACO  JUZGADO DE FAMILIA           AV. LA RIVERA PLAZA SAN MARTIN   
1    DULANTO  JUZGADO DE FAMILIA                                   CALLAO   
2  JICAMARCA               OTROS  Mz. Q  S/N Ovalo Principal de Jicamarca   
3      OCOÑA               OTROS                    AV. PANAMERICA SUR SN   
4    PARCONA  JUZGADO DE FAMILIA                             JJ ELIAS S/N   

               DIST_CIA            DIST_HECHO     DPTO_CIA   DPTO_HECHO  EDAD  \
0             HUANCHACO             HUANCHACO  LA LIBERTAD  LA LIBERTAD    32   
1                CALLAO                CALLAO       CALLAO       CALLAO    40   
2  LURIGANCHO - CHOSICA  LURIGANCHO - CHOSICA         LIMA         LIMA    28   
3                 OCOÑA                 OCOÑA     AREQUIPA     AREQUIPA    55   
4               PARCONA               PARCONA          ICA          ICA    21   

     EST_CIVIL FECHA_HORA_HECHO  ... FEC_REGISTRO 

In [36]:
# Ya no se necesita la columna FEC_REGISTRO ni FECHA_HORA_HECHO
datalake = datalake.drop(['FEC_REGISTRO', 'FECHA_HORA_HECHO'], axis=1)
print(datalake.info())
print(update_types())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406431 entries, 0 to 406430
Data columns (total 29 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   COMISARIA                 406431 non-null  object
 1   DERIVADA_FISCALIA         406431 non-null  object
 2   DIRECCION                 406431 non-null  object
 3   DIST_CIA                  406431 non-null  object
 4   DIST_HECHO                406431 non-null  object
 5   DPTO_CIA                  406431 non-null  object
 6   DPTO_HECHO                406431 non-null  object
 7   EDAD                      406431 non-null  int64 
 8   EST_CIVIL                 406431 non-null  object
 9   LIBRO                     406431 non-null  object
 10  MODALIDAD                 406431 non-null  object
 11  PROV_CIA                  406431 non-null  object
 12  PROV_HECHO                406431 non-null  object
 13  REGION                    406431 non-null  object
 14  SEXO

In [37]:
datalake.to_csv('./dataset/dataset_preprocessed.csv', index=False)

#### Transformación de atributos categóricos a numéricos

In [38]:
from sklearn.preprocessing import LabelEncoder

cat_columns_encoded = {}

def encode(col):
    cat_columns_encoded[col] = LabelEncoder()
    datalake[col] = cat_columns_encoded[col].fit_transform(datalake[col])

for col in object_columns:
    encode(col)

In [39]:
# Función para mostrar el encodificado de una columna
def show_encoded_column(col):
    classes = cat_columns_encoded[col].classes_
    for i, c in enumerate(classes):
        print(f'{i} -> {c}')

In [40]:
show_encoded_column('MODALIDAD')

0 -> AMENAZA GRAVE
1 -> COACCION GRAVE
2 -> MALTRATO SIN LESION
3 -> VIOLENCIA ECONOMICA O PATRIMONIAL
4 -> VIOLENCIA FISICA
5 -> VIOLENCIA FISICA Y PSICOLOGICA
6 -> VIOLENCIA PSICOLOGICA


In [41]:
print(datalake.head())
print(update_types())

   COMISARIA  DERIVADA_FISCALIA  DIRECCION  DIST_CIA  DIST_HECHO  DPTO_CIA  \
0        371                  4        283       252         453        12   
1        311                  4        539        78         130         6   
2        431                  7       1181       376         651        14   
3        598                  7        330       450         772         3   
4        664                  4        872       496         847        10   

   DPTO_HECHO  EDAD  EST_CIVIL  LIBRO  ...  VIA  PAIS_NATAL  \
0          12    32          1      1  ...   10         106   
1           6    40          4     25  ...   10         106   
2          14    28          1     25  ...   10         106   
3           3    55          4     22  ...    4         106   
4          10    21          4     18  ...    3         106   

   FEC_REGISTRO_ANIO  FEC_REGISTRO_MES  FEC_REGISTRO_DIA  \
0               2016                 1                 1   
1               2016            

In [42]:
# LabelEncoder genera el tipo int32
print(datalake.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406431 entries, 0 to 406430
Data columns (total 29 columns):
 #   Column                    Non-Null Count   Dtype
---  ------                    --------------   -----
 0   COMISARIA                 406431 non-null  int32
 1   DERIVADA_FISCALIA         406431 non-null  int32
 2   DIRECCION                 406431 non-null  int32
 3   DIST_CIA                  406431 non-null  int32
 4   DIST_HECHO                406431 non-null  int32
 5   DPTO_CIA                  406431 non-null  int32
 6   DPTO_HECHO                406431 non-null  int32
 7   EDAD                      406431 non-null  int64
 8   EST_CIVIL                 406431 non-null  int32
 9   LIBRO                     406431 non-null  int32
 10  MODALIDAD                 406431 non-null  int32
 11  PROV_CIA                  406431 non-null  int32
 12  PROV_HECHO                406431 non-null  int32
 13  REGION                    406431 non-null  int32
 14  SEXO                

In [43]:
# Trasnformando columnas de tipo int32 a int64
for col in datalake.columns:
    if datalake[col].dtype == 'int32':
        datalake[col] = datalake[col].astype(np.int64)

In [44]:
print(update_types())
print(datalake.info())

object (0)
 [] 
int64 (29)
 ['COMISARIA', 'DERIVADA_FISCALIA', 'DIRECCION', 'DIST_CIA', 'DIST_HECHO', 'DPTO_CIA', 'DPTO_HECHO', 'EDAD', 'EST_CIVIL', 'LIBRO', 'MODALIDAD', 'PROV_CIA', 'PROV_HECHO', 'REGION', 'SEXO', 'SUB_TIPO', 'TIPO', 'TIPO_DENUNCIA', 'UBICACION', 'VIA', 'PAIS_NATAL', 'FEC_REGISTRO_ANIO', 'FEC_REGISTRO_MES', 'FEC_REGISTRO_DIA', 'FEC_REGISTRO_DIA_SEM', 'FECHA_HORA_HECHO_ANIO', 'FECHA_HORA_HECHO_MES', 'FECHA_HORA_HECHO_DIA', 'FECHA_HORA_HECHO_DIA_SEM'] 
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406431 entries, 0 to 406430
Data columns (total 29 columns):
 #   Column                    Non-Null Count   Dtype
---  ------                    --------------   -----
 0   COMISARIA                 406431 non-null  int64
 1   DERIVADA_FISCALIA         406431 non-null  int64
 2   DIRECCION                 406431 non-null  int64
 3   DIST_CIA                  406431 non-null  int64
 4   DIST_HECHO                406431 non-null  int64
 5   DPTO_CIA                  40

In [45]:
print(datalake.head())

   COMISARIA  DERIVADA_FISCALIA  DIRECCION  DIST_CIA  DIST_HECHO  DPTO_CIA  \
0        371                  4        283       252         453        12   
1        311                  4        539        78         130         6   
2        431                  7       1181       376         651        14   
3        598                  7        330       450         772         3   
4        664                  4        872       496         847        10   

   DPTO_HECHO  EDAD  EST_CIVIL  LIBRO  ...  VIA  PAIS_NATAL  \
0          12    32          1      1  ...   10         106   
1           6    40          4     25  ...   10         106   
2          14    28          1     25  ...   10         106   
3           3    55          4     22  ...    4         106   
4          10    21          4     18  ...    3         106   

   FEC_REGISTRO_ANIO  FEC_REGISTRO_MES  FEC_REGISTRO_DIA  \
0               2016                 1                 1   
1               2016            

In [46]:
datalake.to_csv('./dataset/dataset_preprocessed_numeric.csv', index=False)